**Цели и задачи исследования**    
На основании открытых данных Жилищного комитета Санкт-Петербурга провести анализ жилищного фонда, построенного после 1991 года:  
- распределение площади построенного жилого фонда по районам и годам постройки;
- средняя этажность жилищного фонда по районам и годам постройки;
- квартирография (распределение построенных квартир по количеству комнат) по районам и годам постройки.  
Часть 1 анализа посвящена предобработке данных, часть 2 - непосредственно анализу.  
Анализ выполнен на наборе данных "Технико-экономические паспорта многоквартирных домов" (версия № 3 от 03.04.2023):  
https://data.gov.spb.ru/irsi/7840013199-Tehniko-ekonomicheskie-pasporta-mnogokvartirnyh-domov/versions/

In [1]:
import pandas as pd
import warnings
warnings.simplefilter(action = 'ignore', category=Warning)
spb_houses = pd.read_csv('Tehniko-ekonomicheskie-pasporta-mnogokvartirnyh-domov.csv', sep = ',')
spb_houses.head()

,Улица,Дом,Корпус,Литера,Район,Тип коммунальной квартиры,Количество коммунальных квартир,Общее количество комнат в коммунальных квартирах,"Серия, тип проекта",Год постройки,...,Аварийность,Год проведения капитального ремонта,Виды работ,Количество стволов мусоропровода,Площадь металлической кровли,Общее количество лифтов,Количество ПЗУ,"Площадь подвалов, м2",Порядковый номер,Адрес
0,"Г. Красное Село, Красногородская ул.",19,3,А,Красносельский,NaN,NaN,NaN,1-528КП-41,1973,...,0,1997;2004;2010;2012,Очистка и антисептирование древесины;Благоустр...,1.0,NaN,1.0,1.0,239.0,1,"город Красное Село, Красногородская улица, дом..."
1,"Г. Пушкин, Саперная ул.",40,1,З,Пушкинский,NaN,NaN,NaN,Индивидуальный,1996,...,0,2011,Благоустройство придомовой территории,NaN,NaN,NaN,1.0,338.4,2,"город Пушкин, Сапёрная улица, дом 40, корпус 1..."
2,"Г. Ломоносов, Александровская ул.",36б,NaN,А,Петродворцовый,NaN,NaN,NaN,528кп,1966,...,0,2007,Очистка и антисептирование древесины,NaN,NaN,NaN,NaN,700.0,3,"город Ломоносов, Александровская улица, дом 36..."
3,Ворошилова ул.,9,3,А,Невский,NaN,NaN,NaN,NaN,2003,...,0,NaN,NaN,2.0,NaN,2.0,NaN,631.5,4,"муниципальный округ Правобережный, улица Ворош..."
4,"Пос. Песочный, квартал 6-й",123а,NaN,А,Курортный,3 комн.,1,3,NaN,1952,...,0,2011;2020,Обслед.констр.жил.домов;Очистка и антисептиров...,NaN,225.64,NaN,NaN,NaN,5,"посёлок Песочный, квартал 6-й, дом 123а, литера А"


**1. Предобработка данных**  
**1.1. Формируем рабочий датафрейм и приводим типы данных к нужным**

In [2]:
spb_houses.columns

Index(['Улица', 'Дом', 'Корпус', 'Литера', 'Район',
       'Тип коммунальной квартиры', 'Количество коммунальных квартир',
       'Общее количество комнат в коммунальных квартирах',
       'Серия, тип проекта', 'Год постройки', 'Год проведения реконструкции',
       'Общая площадь здания, м2', 'Площадь жилых помещений, м2',
       'Площадь нежилых помещений функционального назначения, м2',
       'Число лестниц', 'Число этажей', 'Количество проживающих',
       'Площадь мансард, м2', 'Центральное отопление', 'Автономная котельная',
       'Печное отопление', 'Центральное горячее водоснабжение',
       'Горячее водоснабжение от газовых колонок',
       'Горячее водоснабжение от дровяных колонок',
       'Центральное электроснабжение', 'Центральное  газоснабжение',
       'Не центральное газоснабжение', 'Мусоропроводы', 'Тип квартиры',
       'Количество квартир',
       'Общее количество комнат в не коммунальных квартирах',
       'Количество встроенных нежилых помещений',
       'Год в

In [3]:
# формируем датафрейм из колонок  исходной таблицы, которые нам нужны
df = spb_houses[['Район', 'Год постройки', 'Общая площадь здания, м2', 'Площадь жилых помещений, м2', 'Число этажей', 'Тип квартиры',
                 'Количество квартир', 'Порядковый номер']]
df.columns = (['region', 'year', 'square_house', 'square_flat', 'floor', 'flat_type',
                 'flat_cnt', 'id'])
df.head()

,region,year,square_house,square_flat,floor,flat_type,flat_cnt,id
0,Красносельский,1973,2228.70,1963.10,9.0,"1 комн.,2 комн.,3 комн.","18,18,9",1
1,Пушкинский,1996,642.20,548.20,2.0,"2 комн.,3 комн.","3,5",2
2,Петродворцовый,1966,5461.90,3490.80,5.0,"1 комн.,2 комн.,3 комн.","27,22,31",3
3,Невский,2003,6042.00,5413.40,10.0,"1 комн.,2 комн.,3 комн.","10,31,39",4
4,Курортный,1952,128.78,128.78,1.0,3 комн.,1,5


In [4]:
df.info()
# перед работой с данными проверим, что они загрузились в датафрейм в нужных нам типах, если нет - приведем к нужным
# колонки region и year нам нужны в строковом исполнении; square_house, square_flat - в числовом типа float; floor - числовой типа int (чтобы избавиться от десятичного представления); flat_type и flat_cnt для начала нужны будут в строковом
# все колонки, кроме region и id, судя по количеству non-null записей, содержат ячейки с NaN значениями, которые в pandas представлены типом float. их нужно предобработать в зависимоти от нужного нам типа данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23105 entries, 0 to 23104
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   region        23105 non-null  object 
 1   year          23101 non-null  object 
 2   square_house  23101 non-null  float64
 3   square_flat   23102 non-null  float64
 4   floor         23102 non-null  float64
 5   flat_type     22725 non-null  object 
 6   flat_cnt      22725 non-null  object 
 7   id            23105 non-null  int64  
dtypes: float64(3), int64(1), object(4)
memory usage: 1.4+ MB


In [5]:
df.isna().any() # действительно, все колонки, кроме region и id, содержат ячейки с NaN значениями

region          False
year             True
square_house     True
square_flat      True
floor            True
flat_type        True
flat_cnt         True
id              False
dtype: bool

In [6]:
# обрабатываем NaN значения
df['year'] = df['year'].fillna('нет данных')
df['square_house'] = df['square_house'].fillna(0)
df['square_flat'] = df['square_flat'].fillna(0)
df['floor'] = df['floor'].fillna(0).astype(int)
df['flat_type'] = df['flat_type'].fillna('нет данных')
df['flat_cnt'] = df['flat_cnt'].fillna('нет данных')

In [7]:
df.info()
# тип данных колонки floor изменился на нужный нам int. нужно проверить, что колонки с типом object содержат только строковые значения

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23105 entries, 0 to 23104
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   region        23105 non-null  object 
 1   year          23105 non-null  object 
 2   square_house  23105 non-null  float64
 3   square_flat   23105 non-null  float64
 4   floor         23105 non-null  int32  
 5   flat_type     23105 non-null  object 
 6   flat_cnt      23105 non-null  object 
 7   id            23105 non-null  int64  
dtypes: float64(2), int32(1), int64(1), object(4)
memory usage: 1.3+ MB


In [8]:
pd.DataFrame({'region': df['region'].apply(type).unique(),
              'year': df['year'].apply(type).unique(),
              'flat_type': df['flat_type'].apply(type).unique(),
              'flat_cnt': df['flat_cnt'].apply(type).unique()})
# проверили - все строки теперь

,region,year,flat_type,flat_cnt
0,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>


**1.2. Фильтрация рабочего датафрейма по годам постройки зданий**

In [9]:
df['year'].unique()

array(['1973', '1996', '1966', '2003', '1952', '1962', '1913', '2008',
       '1971', '1965', '1964', '1967', '1960', '1959', '1953', '1917',
       '1968', '1958', '1995', '1930', '1979', '1951', '1949', '1993',
       '1969', '1963', '1954', '1988', '1955', '1970', '1984', '1977',
       '2007', '1974', '2009', '1916', '1991', '1957', '2004', '1983',
       '1980', '1982', '1914', '2006', '2005', '1972', '1961', '1841',
       '1940', '2010', '1975', 'До 1952', '2015', '2000', '1987', '2002',
       '1976', '1900', '1999', '2012', '1956', '2011', '2013', '2016',
       '2018', '2021', '2020', '2019', '1911', '1904', '1905', '1941',
       '1823', '1909', '1859', '1912', '1847', '1887', '1910', '1896',
       '1879', '1990', '1908', '1806', '1883', '1950', '1870', '1994',
       '1925', '1903', '1836', '1902', '1986', '1868', '1981', '1936',
       '1897', '1886', '1937', '1901', '1873', '1828', '1840', '1915',
       '1860', '1798', '1815', 'До 1917', '1947', '1874', '1846', '1845',


In [10]:
# фильтруем столбец year по годам постройки - нам нужны дома после 1990 года
years = [str(i) for i in range(1991,2024)]
srch_pat = '|'.join(year for year in years)
df['year_filter'] = df['year'].str.contains(srch_pat)
df = df.query("year_filter == True")

In [11]:
df['year'].unique()
#  фильтрация прошла, но в наборе данных остались дома с корпусами постройки до и после 1990 года. удалим их вручную на следующем шаге, чтобы оставить дома исключительно после 1990 года

array(['1996', '2003', '2008', '1995', '1993', '2007', '2009', '1991',
       '2004', '2006', '2005', '2010', '2015', '2000', '2002', '1999',
       '2012', '2011', '2013', '2016', '2018', '2021', '2020', '2019',
       '1994', '1992', '1997', '2014', '2001', '1996,1998', '1991-1994',
       '2017', '1998', '2013,2015', '2003,2004,2005', '2000,2001',
       '2016,2017,2018,2020', '2000-2001', '1997-2001', '2005-2006',
       '2017,2018', '2010,2011', '2001, 2005', '2004,2005', '1995-1996',
       '2001, 2003', '2010, 2012', '1975,1994', '1979-1983, 1997',
       '2010-2012', '1951(лит А1), 1954 (лит А2), 2001 (лит А3)',
       '1980,2013', '2005,2007'], dtype=object)

In [12]:
# повторно фильтруем столбец year
years_drop = ['1975,1994', '1979-1983, 1997', '1951\(лит А1\), 1954 \(лит А2\), 2001 \(лит А3\)', '1980,2013']
srch_pat_2 = '|'.join(year for year in years_drop)
df['year_filter_2'] = df['year'].str.contains(srch_pat_2)
df = df.query("year_filter_2 == False")

In [13]:
df['year'].unique()
# фильтрация прошла успешно

array(['1996', '2003', '2008', '1995', '1993', '2007', '2009', '1991',
       '2004', '2006', '2005', '2010', '2015', '2000', '2002', '1999',
       '2012', '2011', '2013', '2016', '2018', '2021', '2020', '2019',
       '1994', '1992', '1997', '2014', '2001', '1996,1998', '1991-1994',
       '2017', '1998', '2013,2015', '2003,2004,2005', '2000,2001',
       '2016,2017,2018,2020', '2000-2001', '1997-2001', '2005-2006',
       '2017,2018', '2010,2011', '2001, 2005', '2004,2005', '1995-1996',
       '2001, 2003', '2010, 2012', '2010-2012', '2005,2007'], dtype=object)

In [14]:
# т.к. некоторые значения столбца year содержат интервалы через знак тире или запятую, оставим для простоты анализа только одно значение - первый год (начало постройки дома)
df['year'] = df['year'].str.split('-').str.get(0).str.split(',').str.get(0)
df['year'].unique()
# хотя мы используем набор данных от апреля 2023 года, последний год потройки, по которому есть информация по паспортам, - 2021. скорее всего, на дома 2022 и начала 2023 годов постройки еще не составлены паспорта

array(['1996', '2003', '2008', '1995', '1993', '2007', '2009', '1991',
       '2004', '2006', '2005', '2010', '2015', '2000', '2002', '1999',
       '2012', '2011', '2013', '2016', '2018', '2021', '2020', '2019',
       '1994', '1992', '1997', '2014', '2001', '2017', '1998'],
      dtype=object)

**1.3. Приводим данные распределении квартир по количеству комнат к удобному для анализа формату**

In [15]:
# еще раз посмотрим на данные
df.head()
# для дальнейшего анализа столбцы flat_type и flat_cnt удобнее привести к виду с колонками с названиями "1 комн.", "2 комн." и т.д. и значениями в них из колонки flat_cnt

,region,year,square_house,square_flat,floor,flat_type,flat_cnt,id,year_filter,year_filter_2
1,Пушкинский,1996,642.2,548.2,2,"2 комн.,3 комн.","3,5",2,True,False
3,Невский,2003,6042.0,5413.4,10,"1 комн.,2 комн.,3 комн.","10,31,39",4,True,False
7,Курортный,2008,1500.0,1485.0,3,6 комн.,8,8,True,False
23,Пушкинский,1995,6366.8,5340.8,5,"1 комн.,2 комн.,3 комн.,4 комн.","26,40,27,3",24,True,False
29,Колпинский,2003,3513.4,2679.3,5,"1 комн.,2 комн.,3 комн.","17,22,5",30,True,False


In [16]:
# посмотрим на уникальные значения столбца flat_type
df['flat_type'].unique()
# в наших данных есть максимум 7-комнатные квартиры и дома, в которых нет данных по квартирографии

array(['2 комн.,3 комн.', '1 комн.,2 комн.,3 комн.', '6 комн.',
       '1 комн.,2 комн.,3 комн.,4 комн.',
       '1 комн.,2 комн.,3 комн.,4 комн.,5 комн.',
       '1 комн.,2 комн.,3 комн.,4 комн.,7 комн.',
       '2 комн.,3 комн.,4 комн.', '1 комн.,2 комн.',
       '1 комн.,2 комн.,3 комн.,4 комн.,5 комн.,6 комн.',
       '2 комн.,3 комн.,4 комн.,5 комн.', '3 комн.,4 комн.',
       '1 комн.,2 комн.,3 комн.,5 комн.',
       '3 комн.,4 комн.,5 комн.,6 комн.,7 комн.', '1 комн.,3 комн.',
       '1 комн.,2 комн.,3 комн.,4 комн.,5 комн.,7 комн.', '7 комн.',
       '4 комн.,5 комн.,6 комн.',
       '1 комн.,2 комн.,3 комн.,4 комн.,6 комн.',
       '1 комн.,2 комн.,4 комн.',
       '1 комн.,2 комн.,3 комн.,5 комн.,6 комн.', '5 комн.,6 комн.',
       '3 комн.,4 комн.,5 комн.', 'нет данных', '4 комн.', '5 комн.',
       '4 комн.,6 комн.', '3 комн.', '1 комн.', '1 комн.,4 комн.',
       '2 комн.,3 комн.,6 комн.,7 комн.',
       '1 комн.,2 комн.,3 комн.,4 комн.,5 комн.,6 комн.,7 комн.',
       '2 

In [17]:
# напишем конструкцию для формирования нужных нам новых столбцов
# преобразуем строки в столбцах flat_type и flat_cnt в списки и создаем стобцы типа "1 комн.": элемент из столбца flat_cnt с индексом элемента "1 комн." в столбце flat_type
# при этом строки для домов, где данных о квартирографии нет, также заполнятся нулями. но для дальнейшего анализа этот вариант подойдет
df['flat_cnt'] = df['flat_cnt'].str.split(',')
df['flat_type'] = df['flat_type'].str.split(',')

f_types = ['1 комн.','2 комн.','3 комн.','4 комн.','5 комн.','6 комн.','7 комн.']
for f_type in f_types:
  df[f_type] = df.apply(lambda x: x['flat_cnt'][x['flat_type'].index(f_type)] if f_type in x['flat_type'] else 0, axis = 1)

for f_type in f_types:
  df[f_type] = df[f_type].astype(int)

df.head()

,region,year,square_house,square_flat,floor,flat_type,flat_cnt,id,year_filter,year_filter_2,1 комн.,2 комн.,3 комн.,4 комн.,5 комн.,6 комн.,7 комн.
1,Пушкинский,1996,642.2,548.2,2,"[2 комн., 3 комн.]","[3, 5]",2,True,False,0,3,5,0,0,0,0
3,Невский,2003,6042.0,5413.4,10,"[1 комн., 2 комн., 3 комн.]","[10, 31, 39]",4,True,False,10,31,39,0,0,0,0
7,Курортный,2008,1500.0,1485.0,3,[6 комн.],[8],8,True,False,0,0,0,0,0,8,0
23,Пушкинский,1995,6366.8,5340.8,5,"[1 комн., 2 комн., 3 комн., 4 комн.]","[26, 40, 27, 3]",24,True,False,26,40,27,3,0,0,0
29,Колпинский,2003,3513.4,2679.3,5,"[1 комн., 2 комн., 3 комн.]","[17, 22, 5]",30,True,False,17,22,5,0,0,0,0


In [18]:
# теперь можем оставить в датафрейме только те столбцы, которые понадобятся для анализа
df = df[['region', 'year', 'square_house', 'square_flat', 'floor', 'id', '1 комн.', '2 комн.', '3 комн.', '4 комн.', '5 комн.', '6 комн.', '7 комн.']]
df.head()

,region,year,square_house,square_flat,floor,id,1 комн.,2 комн.,3 комн.,4 комн.,5 комн.,6 комн.,7 комн.
1,Пушкинский,1996,642.2,548.2,2,2,0,3,5,0,0,0,0
3,Невский,2003,6042.0,5413.4,10,4,10,31,39,0,0,0,0
7,Курортный,2008,1500.0,1485.0,3,8,0,0,0,0,0,8,0
23,Пушкинский,1995,6366.8,5340.8,5,24,26,40,27,3,0,0,0
29,Колпинский,2003,3513.4,2679.3,5,30,17,22,5,0,0,0,0
